<a href="https://colab.research.google.com/github/PandaBoi/CSLR_with_RL/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import math
import numpy as np
import pandas as pd
from functools import partial
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.utils as utils
import torchvision.transforms as T
from torch.autograd import Variable
import pdb
from itertools import groupby
!pip install easydict
from easydict import EasyDict as ED
device = "cuda" if torch.cuda.is_available() else "cpu"

#maybe will need for later
# !pip install git+https://github.com/enhuiz/phoenix-datasets
# !pip install xmltodict
# from phoenix_datasets import PhoenixVideoTextDataset
 
# from torch.utils.data import DataLoader



In [18]:
def SL_train(clip, label):
  model.train()  
  preds = model(clip.to(device), label[:,:-1].to(device))              
  optim.zero_grad()
  
  preds = F.cross_entropy(preds.view(-1, preds.size(-1)), label[:,1:].view(-1,label[:,1:]),
                         ignore_index=target_pad) #dont know if we need this 
  loss.backward()
  optim.step()
  
  total_loss += loss.data[0]
  if (i + 1) % print_every == 0:
      loss_avg = total_loss / print_every
      print("time = %dm, epoch %d, iter = %d, loss = %.3f,\
      %ds per %d iters" % ((time.time() - start) // 60,\
      epoch + 1, i + 1, loss_avg, time.time() - temp,\
      print_every))
      total_loss = 0
      temp = time.time()

In [19]:

def get_inplanes():
    return [64, 128, 256, 512]


def conv3x3x3(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=1,
                     bias=False)


def conv1x1x1(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                     out_planes,
                     kernel_size=1,
                     stride=stride,
                     bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv3x3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv1x1x1(in_planes, planes)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = conv3x3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = conv1x1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm3d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 block_inplanes,
                 n_input_channels=3,
                 conv1_t_size=7,
                 conv1_t_stride=1,
                 no_max_pool=False,
                 shortcut_type='B',
                 widen_factor=1.0,
                 n_classes=400):
        super().__init__()

        block_inplanes = [int(x * widen_factor) for x in block_inplanes]

        self.in_planes = block_inplanes[0]
        self.no_max_pool = no_max_pool

        self.conv1 = nn.Conv3d(n_input_channels,
                               self.in_planes,
                               kernel_size=(conv1_t_size, 7, 7),
                               stride=(conv1_t_stride, 2, 2),
                               padding=(conv1_t_size // 2, 3, 3),
                               bias=False)
        self.bn1 = nn.BatchNorm3d(self.in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, block_inplanes[0], layers[0],
                                       shortcut_type)
        self.layer2 = self._make_layer(block,
                                       block_inplanes[1],
                                       layers[1],
                                       shortcut_type,
                                       stride=2)
        self.layer3 = self._make_layer(block,
                                       block_inplanes[2],
                                       layers[2],
                                       shortcut_type,
                                       stride=2)
        self.layer4 = self._make_layer(block,
                                       block_inplanes[3],
                                       layers[3],
                                       shortcut_type,
                                       stride=2)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(block_inplanes[3] * block.expansion, n_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _downsample_basic_block(self, x, planes, stride):
        out = F.avg_pool3d(x, kernel_size=1, stride=stride)
        zero_pads = torch.zeros(out.size(0), planes - out.size(1), out.size(2),
                                out.size(3), out.size(4))
        if isinstance(out.data, torch.FloatTensor.to(device)):
            zero_pads = zero_pads.to(device)

        out = torch.cat([out.data, zero_pads], dim=1)

        return out

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(self._downsample_basic_block,
                                     planes=planes * block.expansion,
                                     stride=stride)
            else:
                downsample = nn.Sequential(
                    conv1x1x1(self.in_planes, planes * block.expansion, stride),
                    nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(
            block(in_planes=self.in_planes,
                  planes=planes,
                  stride=stride,
                  downsample=downsample))
        self.in_planes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.in_planes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if not self.no_max_pool:
            x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def generate_model(model_depth, **kwargs):
    assert model_depth in [10, 18, 34, 50, 101, 152, 200]

    if model_depth == 10:
        model = ResNet(BasicBlock, [1, 1, 1, 1], get_inplanes(), **kwargs)
    elif model_depth == 18:
        model = ResNet(BasicBlock, [2, 2, 2, 2], get_inplanes(), **kwargs)
    elif model_depth == 34:
        model = ResNet(BasicBlock, [3, 4, 6, 3], get_inplanes(), **kwargs)
    elif model_depth == 50:
        model = ResNet(Bottleneck, [3, 4, 6, 3], get_inplanes(), **kwargs)
    elif model_depth == 101:
        model = ResNet(Bottleneck, [3, 4, 23, 3], get_inplanes(), **kwargs)
    elif model_depth == 152:
        model = ResNet(Bottleneck, [3, 8, 36, 3], get_inplanes(), **kwargs)
    elif model_depth == 200:
        model = ResNet(Bottleneck, [3, 24, 36, 3], get_inplanes(), **kwargs)

    return model

In [20]:

class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)  # (N, value_len, heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size,)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(
              x.float().unsqueeze(0), 
              # self.word_embedding(x) 
            # + self.position_embedding(positions)
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))
        # x = self.dropout(x+ self.position(position_embedding(positions)))
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out


class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0.3,
        device="cpu",
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        out = F.softmax(out,-1)
        return out



In [21]:

class SC_REINFORCE():
    def __init__(self, opts):
      self.opts = opts
      print(self.opts)
      self.model = Transformer(src_vocab_size = opts.src_vocab_size ,
                                trg_vocab_size = opts.trg_vocab_size,
                                src_pad_idx = opts.src_pad_idx,
                                trg_pad_idx = opts.trg_pad_idx,
                                embed_size=512,
                               max_length = opts.max_len,
                               )
      self.model = self.model.to(device)
      self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
      self.model.train()

    def select_action_sample(self, state):
      #for R
      # print(state['data'],state['gloss'])
      probs = self.model(state['data'],state['gloss'][:,:-1])
      # print(probs.shape, probs.min(),probs.max())       
      action = [probs[:,i,:].multinomial(1).item() for i in range(np.shape(probs)[1]) ]
      # print(action)
      prob = probs[:, :,action].view(1, -1)
      prob = [probs[:,i,a] for i,a in enumerate(action)]
      log_prob = prob[0].log().unsqueeze(0)
      # print('log prob: ',log_prob.shape)
      # entropy = - (probs*probs.log()).sum()
      # print("action: ",action)
      return action, log_prob

    def select_action_argmax(self, state):
      # for R_hat
      probs = self.model(state['data'],state['gloss'][:,:-1])
      # print(probs.shape, probs.min(),probs.max())       
      action = [probs[:,i,:].argmax().item() for i in range(np.shape(probs)[1]) ]
      # print(action)
      prob = probs[:, :,action].view(1, -1)
      # log_prob = [prob[0].log()]
      # print('log_prob: ',log_prob.shape)
      # entropy = - (probs*probs.log()).sum()
      return action
    
    def inference(self,state):
      #[dataset --> ph_video] "ph_video.voc.word2index['<BOS>']" = returns idx of the token

      # inp  = ph_video.voc.word2index['<BOS>']
      inp  = torch.tensor([ph_video.voc.word2index['<BOS>']]).unsqueeze(0).to(device)

      while True:
        # print("getting action")
        action = self.select_action_argmax(state)
        # print("getting next word")
        next_word = np.argmax(action,axis=-1)
        # print(next_word)
        # break
      #   #next_word = probs;
        # print(len(inp),np.shape(next_word))
        inp = torch.cat([inp,torch.tensor([[next_word]])],dim=-1)                 
        
        # break
        if next_word == ph_video.voc.word2index['<EOS>'] or np.shape(inp)[-1] >= self.opts.max_len:
          break
      return inp 
      
    def update_parameters(self, rewards, rewards_argmax, log_probs, gamma = 1.):
      rewards, rewards_argmax = torch.tensor(rewards), torch.tensor(rewards_argmax)
      R = torch.zeros(1, 1)
      loss = 0
      R_hat = rewards_argmax.sum()
      # print(np.shape(log_probs[0]),np.shape())
      for i in reversed(range(len(rewards))):
          R = gamma * R + rewards[i]
          loss = loss - (log_probs[i]*(Variable(R).expand_as(log_probs[i]).to(device) - Variable(R_hat).expand_as(log_probs[i]).to(device)))
      loss = loss / len(rewards)
      # print(loss)
      self.optimizer.zero_grad()
      loss.backward()
      utils.clip_grad_norm(self.model.parameters(), 40)
      self.optimizer.step()


In [22]:
import os
import struct
import logging
import random

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from skimage import io, transform
from PIL import Image

class PhoenixVideo(Dataset):
    def __init__(self, corpus_dir, ANNOT_DIR, video_path, phase, DEBUG=False):
        self.ANNOT_DIR = ANNOT_DIR
        self.vocab_file = os.path.join(ANNOT_DIR,'automatic/newtrainingClasses.txt')
        self.image_type = 'png'
        self.max_video_len = 10000
        self.corpus_dir = corpus_dir
        self.video_path = video_path
        self.phase = phase
        self.alignment = {}
        self.voc = Voc(self.vocab_file)

        self.phoenix_dataset = self.load_video_list()
        # print(self.phoenix_dataset)
        self.data_dict = self.phoenix_dataset[phase]
        if DEBUG == True:
            self.data_dict = self.data_dict[:101]
        logging.info('[DATASET: {:s}]: total {:d} samples.'.format(phase, len(self.data_dict)))

    def __len__(self):
        return len(self.data_dict)

    def __getitem__(self, idx):
        cur_vid_info = self.data_dict[idx]
        id = cur_vid_info['id']
        video_file = cur_vid_info['path']
        label = cur_vid_info['label']
        label_text = cur_vid_info['label_text']
        video_tensor = self.load_video(video_file)
        sample = {'id': id, 'data': video_tensor, 'label': label, 'label_text':label_text}
        if self.phase == 'train':
          glosses = cur_vid_info['gloss'].split(' ')
          # print("The shape of glosses is", np.shape(glosses))
          # print("The shape of data is", np.shape(video_tensor))
          # print("The shape of label is", len(label))
          # print("The shape of label_text is", len(label_text))
          sample['gloss'], sample['gloss_text'] = self.clip_glosses(sample, glosses)
        return sample # final return (data: (num_clips,512), label:(1,len_of_sentence), gloss: (num_clips,10), id:(1))

    def clip_glosses(self, sample, glosses, window_size = 8, skip_size = 4):
      #input shape: (1,num_of_frames_in_video)
      # segment gloss sentence into clip sizes (num_clips,8) [assert len(data) == num_clips]
      # add BOS and EOS to each glossclip (num_clips, 10)
      gloss_text_clips = []
      gloss_clips = []
      j = 0
      for i in range(0, len(glosses) - window_size, skip_size):
        j = j + 1
        gloss_clip = []
        gloss_clip.append('<BOS>')
        gloss_clip.extend(glosses[i:i+window_size])
        gloss_clip.append('<EOS>')
        gloss_clip = [] + gloss_clip
        gloss_clips.append(self.sentence2index(gloss_clip))
        gloss_text_clips.append(gloss_clip)
      gloss_clips = np.array(gloss_clips)
      # gloss_text_clips = np.array(gloss_text_clips)
      print("Len of gloss clips is", np.shape(gloss_clips))
      print("Shape of data is", sample['data'].shape)
      assert sample['data'].shape[0] == np.shape(gloss_clips)[0]
      return gloss_clips, gloss_text_clips

    
    def load_video(self, video_name):
        print('video name: ',video_name)
        feat = caffeFeatureLoader.loadVideoC3DFeature(video_name, 'pool5')
        feat = torch.tensor(feat)
        return feat

    def load_video1(self, video_name):
        frames_list = glob.glob(os.path.join(video_name, '*.{:s}'.format(self.image_type)))
        frames_list.sort()
        num_frame = len(frames_list)
        if self.phase=='train' and self.sample and num_frame > self.max_video_len:
            for _ in range(num_frame-self.max_video_len):
                frames_list.pop(np.random.randint(len(frames_list)))
        frames_tensor_list = [self.load_image(frame_file) for frame_file in frames_list]
        video_tensor = torch.stack(frames_tensor_list, dim=0)
        return video_tensor

    def load_image(self, img_name):
        image = Image.open(img_name)
        image = self.transform(image)
        return image
    
    def load_gloss(self): # TO DO using:- https://github.com/enhuiz/phoenix-datasets/blob/570481bf03a46555ca219f79ace1a2cfab149f8c/phoenix_datasets/corpora.py#L35
        gloss_path = os.path.join(self.ANNOT_DIR,'automatic/train.alignment')
        read = partial(pd.read_csv, sep=" ", na_filter=False)
        ali = read(gloss_path, header=None, names=["id", "classlabel"])
        cls = read(os.path.join(self.ANNOT_DIR,"automatic/trainingClasses.txt"))
        # print(cls)
        df = pd.merge(ali, cls, how="left", on="classlabel")
        # print(df)
        del df["classlabel"]
        # print(df["signstate"])
        df["gloss"] = df["signstate"].apply(lambda s: s.rstrip("012"))

        df["id"] = df["id"].apply(lambda s: "/".join(s.split("/")[3:-2]))
        grouped = df.groupby("id")

        gdf = grouped["gloss"].agg(" ".join)
        sdf = grouped["signstate"].agg(" ".join)

        df = pd.merge(gdf, sdf, "inner", "id")
        # print(df.loc['01April_2010_Thursday_heute_default-0'])
        return df


    def load_video_list(self):
        phoenix_dataset = {}
        outliers = ['13April_2011_Wednesday_tagesschau_default-14'] # '05July_2010_Monday_heute_default-8'
        for task in ['train', 'dev', 'test']:
            if task != self.phase:
                continue
            dataset_path = os.path.join(self.video_path, task)
            corpus = pd.read_csv(os.path.join(self.corpus_dir, '{:s}.corpus.csv'.format(task)), sep='|')
            videonames = corpus['folder'].values
            annotation = corpus['annotation'].values
            if self.phase == 'train':
              glosses_df = self.load_gloss()
              # print(glosses_df.columns)
            ids = corpus['id'].values
            num_sample = len(ids)
            # print('num_sample: ',num_sample)
            video_infos = []
            for i in range(num_sample):
                if ids[i] in outliers:
                    continue
                tmp_info = {
                    'id': ids[i],
                    'path': os.path.join(self.video_path, task, videonames[i].replace('*.png', '')),
                    'label_text': annotation[i],
                    'label': np.array(self.sentence2index(annotation[i].split(' ')))
                }
                if self.phase == 'train':
                  try:
                    tmp_info['gloss'] = glosses_df.loc[ids[i]]['gloss']
                  except:
                    # print('pass')
                    continue
                video_infos.append(tmp_info)
            phoenix_dataset[task] = video_infos
        return phoenix_dataset

    def sentence2index(self, sent):
        #sent = sent.split(' ')
        # print(sent)
        s = []
        for word in sent:
            if word in self.voc.word2index:
                s.append(self.voc.word2index[word])
            else:
                s.append(self.voc.word2index['<UNK>'])
        return s

    def index2sentence(self, indices):
      return [self.voc.index2word[ind] for ind in indices]


class Voc():
    def __init__(self, vocab_file):
        PAD_token = 0
        self.vocab_file = vocab_file
        self.word2index = {'PAD': PAD_token}
        self.index2word = {PAD_token: 'PAD'}
        self.num_words = 1

        count = 0
        with open(self.vocab_file, 'r') as fid:
            for line in fid:
                if count != 0:
                    line = line.strip().split(' ')
                    word = line[0]
                    if word not in self.word2index:
                        self.word2index[word] = self.num_words
                        self.index2word[self.num_words] = word
                        self.num_words += 1
                count += 1
        UNK_token = self.num_words
        BOS_token = self.num_words + 1
        EOS_token = self.num_words + 2
        BLANK_token = self.num_words + 3
        self.word2index['<UNK>'] = UNK_token
        self.word2index['<BOS>'] = BOS_token
        self.word2index['<EOS>'] = EOS_token
        self.word2index['<BLANK>'] = BLANK_token
        self.index2word[UNK_token] = '<UNK>'
        self.index2word[BOS_token] = '<BOS>'
        self.index2word[EOS_token] = '<EOS>'
        self.index2word[BLANK_token] = '<BLANK>'
        self.num_words += 4

class caffeFeatureLoader():
    @staticmethod
    def loadVideoC3DFeature(sample_name, feattype = 'pool5'):
        featnames = glob.glob(os.path.join(sample_name, '*.' + feattype))
        featnames.sort()
        feat = []
        for name in featnames:
            feat.append(caffeFeatureLoader.loadC3DFeature(name)[0])
        return feat

    @staticmethod
    def loadC3DFeature(filename):
        feat = []
        with open(filename, 'rb') as fileData:
            num = struct.unpack("i", fileData.read(4))[0]
            chanel = struct.unpack("i", fileData.read(4))[0]
            length = struct.unpack("i", fileData.read(4))[0]
            height = struct.unpack("i", fileData.read(4))[0]
            width = struct.unpack("i", fileData.read(4))[0]
            blob_shape = [num, chanel, length, height, width]
            m = num * chanel * length * height * width
            for i in range(m):
                val = struct.unpack("f", fileData.read(4))[0]
                feat.append(val)
        return feat, blob_shape



In [39]:
class Environment():
    def __init__(self, ph_dataset ,dataLoader, window_size = 8, skip = 4):
      self.ph_dataset = ph_dataset
      self.dataLoader = dataLoader
      self.window_size = window_size
      self.skip = skip
      self.current_video = None
      self.sample = None
      self.gen = None
      self.actions = []
      self.num_vids = len(self.dataLoader)
    
    def append_a(self, actions, a):
      if len(actions) == 0:
        actions = a
      else:
        np.append(actions, a[int(len(a)/2):])
      actions = np.array(actions).flatten()
      return actions     


    def next_video(self):
      self.sample = next(iter(self.dataLoader))
      #self.current_video, self.label, _ = next(dataLoader)
      self.gen = self.gen_data_and_gloss()
      #frames, _, _ = self.step()
      data,_,_ = self.step()   
      return data   #what do we want to return here?

    def gen_data_and_gloss(self):
      sample = self.sample
      for i in range(sample['data'].shape[1]):
        # print(sample)
        yield {'data': sample['data'][:,i,:], 'gloss': sample['gloss'][:,i,:], 'gloss_text': sample['gloss_text'][i]}


    # def gen_frames_and_reward(self):
    #   video = self.current_video

    #   if video.shape[0] % self.window_size:
    #     raise Exception("Video size not multiple of window size!")
    #     return

    #   for i in range(0,video.shape[0] - self.window_size + 1, self.skip): 
    #       reward = 0
    #       done = False
    #       frames = video[i:i+self.window_size,:,:,:]
    #       if(i >= video.shape[0] - self.window_size): #is last set of frames
    #         self.hyp = action_to_words(self.actions)
    #         reward = get_wer_delsubins(self.ref, self.hyp)
    #         done = True
    #       yield frames, reward, done

    def get_wer(self, ref,actions):
      un_tags = np.where(np.logical_or(actions == self.ph_dataset.voc.word2index['<EOS>'], actions == self.ph_dataset.voc.word2index['<BOS>']))
      actions = np.delete(actions, un_tags)
      hyp_list = [i[0] for i in groupby(self.actions)]
      print(ref,hyp_list)
      ref = ' '.join(e for e in self.ph_dataset.index2sentence(ref.numpy()[0]))
      print('ref: ',ref)
      hyp = ' '.join(e for e in self.ph_dataset.index2sentence(hyp_list))
      wer = get_wer_delsubins(ref, hyp)
      return 1 - wer[0]

    def step(self,action = None):
      done = False
      reward = 0
      # CHANGE: step returns (data:(1,512), gloss:(1,10))
      if action:
        self.actions = self.append_a(self.actions, action)
      #frames, reward, done = next(self.gen, ('', '', ''))
      clip = next(iter(self.gen), None)
     
      if clip == None:
        print('last clip')
        done = True
        self.ref = self.sample['label']
        # hyp_list = [i[0] for i in groupby(self.actions)] 
        # self.hyp = ' '.join([str(e) for e in hyp_list])
        reward = self.get_wer(self.ref, self.actions)
        
      return clip, reward, done


    

In [40]:
# BASE_DIR = "/content/drive/MyDrive/RL_Project_CodeBase"
# PHOE_DIR = os.path.join(BASE_DIR,'phoenix2014-release/phoenix-2014-multisigner')
# FEAT_DIR = os.path.join(BASE_DIR,'c3d_res_phoenix_body_iter5_120k')
# ANNOT_DIR = os.path.join(PHOE_DIR,'annotations')
# CORPUS_DIR = os.path.join(ANNOT_DIR,'manual')
# video_dataset = PhoenixVideo(CORPUS_DIR, ANNOT_DIR, FEAT_DIR, 'train' )
# video_dataset.__len__()

In [43]:
def get_wer_delsubins(ref, hyp, debug=False):
    DEL_PENALTY = 1
    SUB_PENALTY = 1
    INS_PENALTY = 1
    r = ref
    h = hyp
    # costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h) + 1)] for outer in range(len(r) + 1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h) + 1)] for outer in range(len(r) + 1)]

    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3

    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r) + 1):
        costs[i][0] = DEL_PENALTY * i
        backtrace[i][0] = OP_DEL

    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS

    # computation
    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                costs[i][j] = costs[i - 1][j - 1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i - 1][j - 1] + SUB_PENALTY  # penalty is always 1
                insertionCost = costs[i][j - 1] + INS_PENALTY  # penalty is always 1
                deletionCost = costs[i - 1][j] + DEL_PENALTY  # penalty is always 1

                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL

    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i -= 1
            j -= 1
            if debug:
                lines.append("OK\t" + r[i] + "\t" + h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub += 1
            i -= 1
            j -= 1
            if debug:
                lines.append("SUB\t" + r[i] + "\t" + h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j -= 1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i -= 1
            if debug:
                lines.append("DEL\t" + r[i] + "\t" + "****")
    if debug:
        lines = reversed(lines)
        for line in lines:
            print(line)
        print("#cor " + str(numCor))
        print("#sub " + str(numSub))
        print("#del " + str(numDel))
        print("#ins " + str(numIns))
    return (numSub + numDel + numIns) / (float)(len(r)), numSub / float(len(r)), numIns / float(len(r)), numDel / float(len(r))

In [44]:

BASE_DIR = "/content/drive/MyDrive/RL_Project_CodeBase"
PHOE_DIR = os.path.join(BASE_DIR,'phoenix2014-release/phoenix-2014-multisigner')
FEAT_DIR = os.path.join(BASE_DIR,'c3d_res_phoenix_body_iter5_120k')
ANNOT_DIR = os.path.join(PHOE_DIR,'annotations')
CORPUS_DIR = os.path.join(ANNOT_DIR,'manual')


# env = ENV()
# helper function append_action(actions, a): returns appended action after ignoring 4 overlap frames
video_dataset = PhoenixVideo(CORPUS_DIR, ANNOT_DIR, FEAT_DIR, 'train' )
dataLoader = DataLoader(video_dataset,
    batch_size=1, shuffle=False)

env = Environment(video_dataset, dataLoader)
env2 = Environment(video_dataset, dataLoader)
opts = ED({
    "src_vocab_size": 512,
    "trg_vocab_size": video_dataset.voc.num_words,
    "src_pad_idx": 0,
    "trg_pad_idx": 0,
    "embed_size" : 512,
    "num_layers": 6,
    "forward_expansion": 4,
    "heads": 8,
    "dropout": 0.3,
    "device": "cuda" if torch.cuda.is_available() else "cpu", 
    "max_len": 10,
})

agent = SC_REINFORCE(opts) # add args 
train_wers = []
dev_wers = []
test_wers = []


{'src_vocab_size': 512, 'trg_vocab_size': 1237, 'src_pad_idx': 0, 'trg_pad_idx': 0, 'embed_size': 512, 'num_layers': 6, 'forward_expansion': 4, 'heads': 8, 'dropout': 0.3, 'device': 'cpu', 'max_len': 10}


In [ ]:

# x = torch.tensor(np.random.randn(1,512))
# trg = torch.tensor([np.arange(10)]*1)

# print(np.shape(trg))
# trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)
# src_pad_idx = 0
# trg_pad_idx = 0
# src_vocab_size = 10
# trg_vocab_size = 10
# model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
#     device
# )
# out = model(x, trg[:, :-1])
# print(np.shape(out))
for vid_num in range(env.num_vids): #env.num_epochs
  state = env.next_video()
  state2 = env2.next_video()
  # state = {'clip': x, 'label': trg}
  # vid = [state]

  reward_s = []
  reward_a = []
  log_probs = []
  done_s = done_a = False

  while not done_s or not done_a:
    
    if not done_s:
      a_s, log_prob = agent.select_action_sample(state)
        # done_s = True #debug - remove later, env step
        # r_s = [1*9] #debug - remove later, env step
        #state, reward, done = env.step(action)
      print("a_s: ",a_s)
      state, r_s, done_s = env.step(a_s)
      reward_s.append(r_s)
      log_probs.append(log_prob)
   
    if not done_a:
      a_a = agent.select_action_argmax(state2)
        # done_a = True #debug - remove later, env step
        # r_a = [1*10] #debug - remove later, env step
      state2, r_a, done_a = env2.step(a_a)
      reward_a.append(r_a)

    if done_a and done_s:
      break
      
  reward_a, reward_s, log_probs = np.array(reward_a), np.array(reward_s), np.array(log_probs)
  agent.update_parameters(reward_s, reward_a,log_probs=log_probs)


# agent.inference(state)






In [ ]:
with torch.no_grad():
  agent.inference(state)